# Incomes

In [1]:
import requests
import pandas as pd
import json
import time
from datetime import date

In [ ]:
# Склад остатки - stocks?dateFrom=2023-04-12
# Заказы        - orders?dateFrom=2023-02-09&flag=0
# Продажи       - sales?dateFrom=2023-02-09&flag=0
# Поставки      - incomes?dateFrom=2023-03-19
# Отчет о продажах по реализации - reportDetailByPeriod?dateFrom=2023-02-09&dateTo=2023-02-28

In [2]:
# Выгружаем данные
# На выходе получается строка с информацие в виде словаря
url = "https://statistics-api.wildberries.ru/api/v1/supplier/incomes?dateFrom=2023-01-01"
payload = {}
headers = {
  'Authorization': ''
}
response = requests.request("GET", url, headers=headers, data=payload)
print(response.text)

[{"incomeId":10960235,"number":"","date":"2023-02-21T00:00:00","lastChangeDate":"2023-02-21T13:46:34.13","supplierArticle":"Синий халат","techSize":"XS-L","barcode":"2037488779201","quantity":37,"totalPrice":0,"dateClose":"2023-02-21T00:00:00","warehouseName":"Подольск","nmId":147263461,"status":"Принято"},{"incomeId":10960235,"number":"","date":"2023-02-21T00:00:00","lastChangeDate":"2023-02-21T13:46:34.13","supplierArticle":"Этнический халат","techSize":"XS-L","barcode":"2037488839493","quantity":28,"totalPrice":0,"dateClose":"2023-02-21T00:00:00","warehouseName":"Подольск","nmId":147263932,"status":"Принято"},{"incomeId":10960235,"number":"","date":"2023-02-21T00:00:00","lastChangeDate":"2023-02-21T13:46:34.13","supplierArticle":"Шелковая пижама лист1","techSize":"XS-L","barcode":"2037488886244","quantity":25,"totalPrice":0,"dateClose":"2023-02-21T00:00:00","warehouseName":"Подольск","nmId":147271346,"status":"Принято"},{"incomeId":10960235,"number":"","date":"2023-02-21T00:00:00","

In [3]:
# Данные в переменную
str_total = response.text  # достаём данные в строку
str_total

'[{"incomeId":10960235,"number":"","date":"2023-02-21T00:00:00","lastChangeDate":"2023-02-21T13:46:34.13","supplierArticle":"Синий халат","techSize":"XS-L","barcode":"2037488779201","quantity":37,"totalPrice":0,"dateClose":"2023-02-21T00:00:00","warehouseName":"Подольск","nmId":147263461,"status":"Принято"},{"incomeId":10960235,"number":"","date":"2023-02-21T00:00:00","lastChangeDate":"2023-02-21T13:46:34.13","supplierArticle":"Этнический халат","techSize":"XS-L","barcode":"2037488839493","quantity":28,"totalPrice":0,"dateClose":"2023-02-21T00:00:00","warehouseName":"Подольск","nmId":147263932,"status":"Принято"},{"incomeId":10960235,"number":"","date":"2023-02-21T00:00:00","lastChangeDate":"2023-02-21T13:46:34.13","supplierArticle":"Шелковая пижама лист1","techSize":"XS-L","barcode":"2037488886244","quantity":25,"totalPrice":0,"dateClose":"2023-02-21T00:00:00","warehouseName":"Подольск","nmId":147271346,"status":"Принято"},{"incomeId":10960235,"number":"","date":"2023-02-21T00:00:00",

In [4]:
# Создаём первую строку датафрэйма
# Убираем лишние символы, разбиваем, берём первый заказ, добавляем символы для словаря
s_first = '{' + str_total.replace('[', '').replace(']', '').split('},{')[0].replace('{', '').replace('}', '') + '}'
# преобразуем словарь
json_acceptable_string = s_first.replace('"', "\"")
data = json.loads(json_acceptable_string)
# словарь в дф
df = pd.DataFrame([data])
df

,incomeId,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,dateClose,warehouseName,nmId,status
0,10960235,,2023-02-21T00:00:00,2023-02-21T13:46:34.13,Синий халат,XS-L,2037488779201,37,0,2023-02-21T00:00:00,Подольск,147263461,Принято


In [5]:
# Проходимся циклом по строке и дополняем датафрэйм
# Запускать 1 раз!
str_need = str_total.split('},{')
for i in str_need[1:]:  # Со второго элемента, так как первый уже закинул в дф
    i = '{' + i.replace('[', '').replace(']', '').replace('{', '').replace('}', '') + '}'
    json_acceptable_string = i.replace('"', "\"")
    data = json.loads(json_acceptable_string)
    df_t = pd.DataFrame([data])
    df = pd.concat([df, df_t])
df = df.reset_index(drop=True)
df.head(3)

,incomeId,number,date,lastChangeDate,supplierArticle,techSize,barcode,quantity,totalPrice,dateClose,warehouseName,nmId,status
0,10960235,,2023-02-21T00:00:00,2023-02-21T13:46:34.13,Синий халат,XS-L,2037488779201,37,0,2023-02-21T00:00:00,Подольск,147263461,Принято
1,10960235,,2023-02-21T00:00:00,2023-02-21T13:46:34.13,Этнический халат,XS-L,2037488839493,28,0,2023-02-21T00:00:00,Подольск,147263932,Принято
2,10960235,,2023-02-21T00:00:00,2023-02-21T13:46:34.13,Шелковая пижама лист1,XS-L,2037488886244,25,0,2023-02-21T00:00:00,Подольск,147271346,Принято


In [6]:
# Создали колонку с датой
df = df.assign(date = df.dateClose.apply(lambda x: x.split('T')[0]))

In [7]:
# Оставим нужные колонки
df = df[['incomeId', 'date', 'supplierArticle', 'techSize', 'barcode', 'quantity', 'warehouseName', 'nmId', 'status']]

In [8]:
# Дату в тип с датой
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

In [12]:
# Сохранили сегодняшней датой
today = date.today()
df.to_excel(f'incomes_{today}.xlsx', index=False)

# Анализ

In [24]:
# Количество поставленных товаров
total_incomes = df.groupby('supplierArticle', as_index=False).agg({'quantity': 'sum'})  \
    .rename(columns={'quantity': 'quantity_total'}).sort_values('quantity_total', ascending=False)
total_incomes['prc_procure'] = round(total_incomes.quantity_total / total_incomes.quantity_total.sum() * 100)
total_incomes

,supplierArticle,quantity_total,prc_procure
9,Этнический халат,95,40.0
1,Голубалдеж,41,17.0
3,Синий халат,37,16.0
8,Шелковая пижама лист1,25,11.0
0,Большие размеры,14,6.0
7,Шелковая пижама белые листья,5,2.0
5,Цвет ЗАРА,4,2.0
6,Черно-белый,4,2.0
11,черный журавли,4,2.0
2,Красный,2,1.0


In [25]:
# Количество поставленных товаров в файл
today = date.today()
total_incomes.to_excel(f'Количество поставленных товаров_{today}.xlsx', index=False)